# Speaker metric studies : 


In [36]:
import json
from operator import itemgetter

In [37]:
refPath = "/Users/amira/Desktop/corpus_stt/medias_corrected/fr-FR_news_0120be6c-7b6d-447a-bae2-d79a9149ddd6_Journal-Polynesie/reference_transcript_to_evaluate.json"
hypPath = "/Users/amira/Desktop/corpus_stt/medias_corrected/fr-FR_news_0120be6c-7b6d-447a-bae2-d79a9149ddd6_Journal-Polynesie/vocapia/nlp_transcript_norm.json"

In [38]:
#method to read a transcript json file
def read_transcript(transcriptPath):
    try:
        return json.load(open(transcriptPath,'rt',encoding='utf-8'))
    except FileNotFoundError:
        return None

# Study 1
## Table de correspondances des speaker avec leurs intervals de temps de parole

In [115]:
#method to get the speaker time intervals
def get_speaker_screen_time(transcript):
    speakers = transcript['speakers']
    words = transcript['words']
    speakerScreenTimeList = []
    screenTimesList = []
    
    for speaker in speakers : 

        speakerId = speaker['id']
        speakerScreenTime =[]
        
        interList = []
        i = 0
        while i < len(words):
            interList = []
            while i < len(words) and words[i]['speakerId'] == speakerId:
                interList.append(words[i])
                i+=1
            
            if len(interList) > 0 :
                screenTime = {
                    "startTime" : interList[0]['startTime'],
                    "endTime" : interList[-1]['endTime']
                }
                speakerScreenTime.append(screenTime)
                
                newElem =  {
                                "startTime" :  interList[0]['startTime'],
                                "endTime" : interList[-1]['endTime'],
                                "speakerId" : speakerId
                            }
                if not any(d['startTime'] == newElem['startTime'] for d in screenTimesList):
                    screenTimesList.append(newElem)
                
                interList = []
            else :
                i+=1

        
        speaker['speakerScreenTime'] = speakerScreenTime
        speakerScreenTimeList.append(speaker)
                    
    newlist = sorted(screenTimesList, key=itemgetter('startTime')) 
    return speakerScreenTimeList, newlist
           

In [202]:
def creen_screentimes_crossed(hypScreenTime, refScreenTime):
    candidat_speaker= {}
    for hypElem in hypScreenTime:
        startTime = hypElem['startTime']
        endTime = hypElem['endTime']
        speaker = hypElem['speakerId']
        potential_speakers =  []
        for refElem in refScreenTime:
            if refElem['startTime'] > endTime or refElem['endTime'] < startTime :
                continue
                print("___________OUT OF RANGE___________")
                print(speaker, startTime, endTime)
                print(refElem)
            else:
                
                intersection_length = min(float(endTime), float(refElem['endTime'])) - max(float(startTime), float(refElem['startTime']))
                interval_length = float(endTime) - float(startTime)
                
                if not any(d['speakerId'] == refElem["speakerId"] for d in potential_speakers):
                    potential_speakers.append(
                            {
                                "speakerId" : refElem["speakerId"],
                                "ratio" : round(intersection_length/interval_length,2),
                                "intersection_time" : round(intersection_length,2),
                                "interval_length" : round(interval_length,2)
                            }
                    )
                else:

                    potential_speakers = [                          
                            {
                                "speakerId" : refElem["speakerId"],
                                "ratio" : round(intersection_length/interval_length,2) + round(elem['ratio'],2) ,
                                "intersection_time" : round(intersection_length,2) + round(elem['intersection_time'],2),
                                "interval_length" : round(interval_length,2)
                            } if elem['speakerId'] == refElem["speakerId"] else elem for elem in potential_speakers
                    ]
                    

        if candidat_speaker.get(speaker) == None :
            candidat_speaker[speaker] = potential_speakers
        else:
            candidat_speaker[speaker] = candidat_speaker[speaker] + potential_speakers
    return candidat_speaker

                
                
            

In [203]:
def top_speaker(candidat_speaker):
    for key in candidat_speaker:
        spk_dict= {}
        print("speaker",key)
        for elem in candidat_speaker[key]:
            if not(elem['speakerId'] in spk_dict):
                spk_dict[elem['speakerId']] = elem
            else:
                spk_dict[elem['speakerId']]['intersection_time'] += round(elem['intersection_time'],2)
                spk_dict[elem['speakerId']]['interval_length'] += round(elem['interval_length'],2)
                spk_dict[elem['speakerId']]['ration'] = round(
                        (spk_dict[elem['speakerId']]['intersection_time'] + elem['intersection_time'])/
                        (spk_dict[elem['speakerId']]['interval_length'] + elem['interval_length'])
                ,2)
        print(spk_dict)


# TEST AREA

In [204]:
hypTranscript = read_transcript(hypPath)
refTranscript = read_transcript(refPath)

In [205]:
hypSpeakerScreenTime, hypScreenTime = get_speaker_screen_time(hypTranscript)
refSpeakerScreenTime, refScreenTime = get_speaker_screen_time(refTranscript)

In [214]:
print(hypSpeakerScreenTime)

[{'id': 1, 'gender': 'F', 'speakerScreenTime': [{'startTime': 9.31, 'endTime': 98.07000000000001}, {'startTime': 1523.35, 'endTime': 1525.99}]}, {'id': 2, 'gender': 'F', 'speakerScreenTime': [{'startTime': 98.16, 'endTime': 126.4}, {'startTime': 324.36, 'endTime': 331.59}, {'startTime': 337.78, 'endTime': 380.90999999999997}]}, {'id': 3, 'gender': 'M', 'speakerScreenTime': [{'startTime': 127.17, 'endTime': 163.82}]}, {'id': 4, 'gender': 'M', 'speakerScreenTime': [{'startTime': 164.74, 'endTime': 165.56}, {'startTime': 914.03, 'endTime': 920.87}]}, {'id': 5, 'gender': 'F', 'speakerScreenTime': [{'startTime': 166.02, 'endTime': 204.17999999999998}]}, {'id': 6, 'gender': 'F', 'speakerScreenTime': [{'startTime': 206.1, 'endTime': 217.10999999999999}, {'startTime': 240.26, 'endTime': 249.89}, {'startTime': 273.55, 'endTime': 285.33}, {'startTime': 297.64, 'endTime': 299.5}, {'startTime': 309.08, 'endTime': 322.69}, {'startTime': 486.47, 'endTime': 511.21999999999997}]}, {'id': 7, 'gender': 

In [212]:
candidat_speaker = creen_screentimes_crossed(hypScreenTime, refScreenTime)
print(candidat_speaker[1])

[{'speakerId': '1', 'ratio': 1.0, 'intersection_time': 88.76, 'interval_length': 88.76}, {'speakerId': '38', 'ratio': 0.91, 'intersection_time': 2.4, 'interval_length': 2.64}]


In [207]:
top_speaker(candidat_speaker)

speaker 1
{'1': {'speakerId': '1', 'ratio': 1.0, 'intersection_time': 88.76, 'interval_length': 88.76}, '38': {'speakerId': '38', 'ratio': 0.91, 'intersection_time': 2.4, 'interval_length': 2.64}}
speaker 2
{'1': {'speakerId': '1', 'ratio': 1.0, 'intersection_time': 78.6, 'interval_length': 78.6, 'ration': 1.0}}
speaker 3
{'2': {'speakerId': '2', 'ratio': 1.0, 'intersection_time': 36.65, 'interval_length': 36.65}}
speaker 4
{'1': {'speakerId': '1', 'ratio': 0.88, 'intersection_time': 0.72, 'interval_length': 0.82}, '20': {'speakerId': '20', 'ratio': 1.0, 'intersection_time': 6.84, 'interval_length': 6.84}}
speaker 5
{'1': {'speakerId': '1', 'ratio': 1.0, 'intersection_time': 38.16, 'interval_length': 38.16}}
speaker 6
{'3': {'speakerId': '3', 'ratio': 1.0, 'intersection_time': 45.9, 'interval_length': 46.03, 'ration': 1.0}, '8': {'speakerId': '8', 'ratio': 1.0, 'intersection_time': 1.86, 'interval_length': 1.86}, '14': {'speakerId': '14', 'ratio': 1.0, 'intersection_time': 24.75, 'inte

# Study 2 : 
## Detection des changements de speaker 

In [32]:
#method to get an element on speakers collection based on its id to extract the gender
def get_speaker_gender(speakerId, speakers):
    for speaker in speakers:
        if "gender" in speaker and speaker["id"] == speakerId:
            return speaker["gender"]
    return None

In [33]:
#method to get the exact time where a change of speaker is detected regardless of the speaker id
def get_speaker_change_detection(transcript, mode = "speaker"):
    
    words = transcript['words']
    speakers = transcript["speakers"]
    speakerChangeTimes = []
    
    if mode == "speaker":
        i = 0
        actualSpeaker = words[0]['speakerId']

        while i < len(words) :
            if actualSpeaker != words[i]['speakerId']:
                speakerChangeTimes.append(words[i]['startTime'])
                actualSpeaker = words[i]['speakerId']

            i+=1
    elif mode == "gender":
        i = 0
        actualGender = get_speaker_gender(words[0]['speakerId'], speakers)
        while i < len(words) :
            if actualGender != get_speaker_gender(words[i]['speakerId'], speakers):
                speakerChangeTimes.append(words[i]['startTime'])
                actualGender = get_speaker_gender(words[i]['speakerId'],speakers)

            i+=1
        
        
    else:
        return None

    return speakerChangeTimes
    

In [34]:
#method to compare two speaker change time
def compare_speaker_changes(refSpeakerChangeTimes, hypSpeakerChangeTimes):
    confTime = float(0.5)
    
    matches = 0
    
    for refChangeTime in refSpeakerChangeTimes : 
        for hypChangeTime in hypSpeakerChangeTimes :
            
            if float(refChangeTime) >= float(hypChangeTime - confTime) and float(refChangeTime) >= float(hypChangeTime + confTime):
                matches += 1
                hypSpeakerChangeTimes.remove(hypChangeTime)
                break 

    errorRate = (len(refSpeakerChangeTimes) - matches) / len(refSpeakerChangeTimes)
    return errorRate
    
    
    
    